In [20]:
import numpy as np
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
import cv2
import pywt
from sklearn import preprocessing
# import pysteg


def get_feature_vector(img_path):
    '''
        return the extracted feature vector by applying
        a High Order Statistical Steganalysis (Farid Technique) on RGB seperated channels
    '''
    img = cv2.imread(img_path) # read image using opencv
    
    # copy file 3 times for each colour channel (red, green, blue)
    img_red = img.copy()
    img_green = img.copy()
    img_blue = img.copy()

    # isolate colour channels
    img_red[:, :, 1] = 0
    img_red[:, :, 2] = 0
    img_green[:, :, 0] = 0
    img_green[:, :, 2] = 0
    img_blue[:, :, 0] = 0
    img_blue[:, :, 1] = 0
        
    # get features vectors for each colour channel
    colour_channels = [img_red, img_green, img_blue]
    feature_vector = []
    for channel in colour_channels:
        feature_vector.append(farid36(channel))
    return feature_vector


def farid36(I, name="qmf9",*a,**kw):
    """
    Calculate the 36 Farid features from the image I
    Optionally a wavelet name can be given as well.
    """
    wavelet = get_wavelet(name) 
    H = pywt.wavedec2( I, wavelet, level=4 )
#     print(H)
#     return H
    return farid36aux(H)


def farid36aux(H):
    R = []
    for h in H[2:]:
        for A in h:
#             print("AAAAAAAAAAAAAA")
#             print(skew(A,axis=None))
            R.extend([np.mean(A), np.var(A), skew(A, axis=None), kurtosis(A, axis=None)])
#             R.extend([np.mean(A), np.var(A)])
    return R

def get_wavelet(name = "qmf9"):
    ''' Making a QMF-9 wavelet object in Python '''
    lf = [ 0.02807382, -0.060944743, -0.073386624, 0.41472545, 0.7973934,
            0.41472545, -0.073386624, -0.060944743, 0.02807382, 0 ]
    size = len(lf)
    size2 = np.ceil(float(size)/2) - 1
    hf = []
#     for i in range(size-1,-1,-1):
#         hf.append(lf[i]*(-1)**(i-size2))
    hf = [ lf[i]*(-1)**(i-size2) for i in range(size-1,-1,-1) ]
    f = ( lf, hf, list(reversed(lf)), list(reversed(hf)) )
    qmf9 = pywt.Wavelet(name, f)
    return qmf9

# get_wavelet()
# get_feature_vector("Lionel_Messi_512_769.jpg")
# scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
# # x = scaler.fit_transform(x)
# x

In [21]:
# x = get_feature_vector("Lionel_Messi_512_769.jpg")
# x

In [22]:
# import sys
# !{sys.executable} -m pip install pysteg1.0.tar.gz